# Import and Loading Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import module, secret_keys
from model_list import models
import pandas as pd

hf_api_key             = secret_keys.HF_TOKEN                   #<insert your own huggingface token here>
openai_api_key         = secret_keys.OPENAI_API_KEY_TEAM        #<insert your own openai token here>

# Load Dataset

These three IDs are three-shot example IDs in each dataset - ['NCT00000620', 'NCT01483560', 'NCT04280783']. Do not run generation or evaluation on them.

In [19]:
data_pub = pd.read_csv('data/CT-Pub-With-Examples.csv')
print(data_pub.shape)
data_pub.head(2)

(103, 9)


,NCTId,BriefTitle,EligibilityCriteria,BriefSummary,Conditions,Interventions,PrimaryOutcomes,BaselineMeasures,Paper_BaselineMeasures
0,NCT02572882,Gut Microbiome and p-Inulin in Hemodialysis,Inclusion Criteria:\n\n* Maintenance hemodialy...,"The Microbiome trial is a non-randomized, open...","End-Stage Renal Disease, Gut Microbiome Dysbio...","p-inulin,",Within Participant Variability in Microbiome C...,"Age, Categorical, Age, Continuous, Sex: Female...","Age, Sex, Race, Ethnicity, Hypertension, Diabe..."
1,NCT02623348,Use of Pedometers to Measure and Increase Walk...,Inclusion Criteria:\n\n* on hemodialysis for ≥...,Randomized controlled trial using pedometers t...,"End-stage Renal Disease,","pedometer,","Physical Activity,","Age, Continuous, Sex: Female, Male, Ethnicity ...","Age, Sex, Race, Ethnicity, BMI, HTN, DM, CAD, ..."


In [20]:
data_repo = pd.read_csv('data/CT-Repo-With-Examples-Processed-Version.csv')
print(data_repo.shape)
data_repo.head(2)

(1693, 8)


,NCTId,BriefTitle,EligibilityCriteria,BriefSummary,Conditions,Interventions,PrimaryOutcomes,BaselineMeasures
0,NCT02151643,Study to Evaluate the Efficacy and Safety of P...,Inclusion Criteria:\n\n* Men or women aged 18 ...,The main purpose of this study is to see wheth...,"Hyperphosphataemia,","PT20, Placebo,",Change in Serum Phosphate Concentration From B...,"Age, Continuous, Sex: Female, Male, Ethnicity ..."
1,NCT03036150,A Study to Evaluate the Effect of Dapagliflozi...,Inclusion Criteria:\n\n* Provision of signed i...,The purpose of this study is to evaluate the e...,"Chronic Kidney Disease,","Dapagliflozin, Placebo,",Time to the First Occurrence of Any of the Com...,"Age, Continuous, Age, Customized, Age, Customi..."


## Example of Generation using GPT-4o with 0-shot prompts (CT-Pub)

In [21]:
model_name   = models['gpt-4o']

row = next(data_pub.iterrows())[1]
system_message, question = module.build_zeroshot_prompt(data_pub, row)
model_query = module.system_user_template(system_message, question)
model_response = module.run_generation_single_openai(model_query, model_name, 
                                                openai_api_key, temperature=0.0)
print(system_message)
print(question)
print(model_response)


You are a helpful assistant with experience in the clinical domain and clinical trial design.     You'll be asked queries related to clinical trials. These inquiries will be delineated by a '##Question' heading.     Inside these queries, expect to find comprehensive details about the clinical trial structured within specific subsections,     indicated by '<>' tags. These subsections include essential information such as the trial's title, brief summary,     condition under study, inclusion and exclusion criteria, intervention, and outcomes.In answer to this question, return a list of probable baseline features (separated by commas) of the clinical trial.     Baseline features are the set of baseline or demographic characteristics that are assessed at baseline and used in the analysis of the     primary outcome measure(s) to characterize the study population and assess validity. Clinical trial-related publications typically     include a table of baseline features assessed  by arm or co

## Example of Generation using llama3-70b-it with 3-shot prompts

In [22]:
model_hf_endpoint = models['llama3-70b-it']

row = next(data_pub.iterrows())[1]
system_message, question = module.build_three_shot_prompt(data_pub, row)
model_query = module.system_user_template(system_message, question)
model_response = module.run_generation_single_hf_models(model_query, model_hf_endpoint, 
                                                hf_api_key, temperature=0.0)
print(system_message)
print(question)
print(model_response)


You are a helpful assistant with experience in the clinical domain and clinical trial design.     You'll be asked queries related to clinical trials. These inquiries will be delineated by a '##Question' heading.     Inside these queries, expect to find comprehensive details about the clinical trial structured within specific subsections,     indicated by '<>' tags. These subsections include essential information such as the trial's title, brief summary,     condition under study, inclusion and exclusion criteria, intervention, and outcomes.In answer to this question, return a list of probable baseline features (separated by commas) of the clinical trial.     Baseline features are the set of baseline or demographic characteristics that are assessed at baseline and used in the analysis of the     primary outcome measure(s) to characterize the study population and assess validity. Clinical trial-related publications typically     include a table of baseline features assessed  by arm or co

# Example of Eval using GPT-4o

In [27]:
row = data_pub.iloc[1]
reference_list = row['Paper_BaselineMeasures']

#<insert any list generated by any LLM here>
candidate_list = 'age, sex, body mass index, dialysis duration, comorbidities, walking ability, cognitive function, depression score, quality of life score.'

qstart = module.get_question_from_row(row)
system_message, question = module.build_gpt4_eval_prompt(module.extract_elements_with_cleaning(reference_list), 
                                                         module.extract_elements_with_cleaning(candidate_list), 
                                                         qstart)
model_response = module.run_evaluation_with_gpt4o(system_message, question, openai_api_key)

print(system_message)
print(question)
print(model_response)


        You are an expert assistant in the medical domain and clinical trial design. You are provided with details of a clinical trial.
        Your task is to determine which candidate baseline features match any feature in a reference baseline feature list for that trial. 
        You need to consider the context and semantics while matching the features.

        For each candidate feature:

            1. Identify a matching reference feature based on similarity in context and semantics.
            2. Remember the matched pair.
            3. A reference feature can only be matched to one candidate feature and cannot be further considered for any consecutive matches.
            4. If there are multiple possible matches (i.e. one reference feature can be matched to multiple candidate features or vice versa), choose the most contextually similar one.
            5. Also keep track of which reference and candidate features remain unmatched.

        Once the matching is complete, p